In [2]:
# import required libriaries
from pathlib import Path
from helpers import *

In [3]:
# path to data directory
#path = Path('/home/jupyter/fpl-prediction/data')
path = Path('./data')

In [4]:
# read in latest training set
df_train = pd.read_csv(path/'train.csv', index_col=0, dtype={'season':str})

# path to current season
season_paths = [path/'2019-20']

# name for current season
season_names = ['1920']

# team codes
teams = pd.read_csv(path/'teams.csv')

# latest remaining season dataset, for latest week's play_proba values
remaining_season = pd.read_csv(path/'remaining_season.csv', index_col=0)

# create players dataset
all_players = build_players(path, season_paths, season_names, teams)

# 2019-20 market value
# table was in slightly different position for 2019-20 season
# should check that 2019-20 season values remain the same once season starts
teams_mv = build_season_mv('1920', 11, range(13,33))

In [5]:
df_train.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
82802,Heurelho da Silva_Gomes,26,1,0,Watford,Brighton and Hove Albion,0.476769,0.467411,False,0,...,1,1,0.0,-60,112,172,0,2020-02-08T17:30:00Z,1920,1.0
82803,Joseph_Anang,26,1,0,West Ham United,Manchester City,0.758450,2.653289,False,0,...,0,2,0.0,132,529,397,0,2020-02-19T19:30:00Z,1920,1.0
82804,Erik_Pieters,26,2,0,Burnley,Southampton,0.402621,0.452499,False,0,...,2,1,0.0,-8007,617,8624,0,2020-02-15T12:30:00Z,1920,1.0
82805,Japhet_Tanganga,26,2,0,Tottenham Hotspur,Aston Villa,1.845990,0.537651,False,0,...,3,2,0.0,68650,74632,5982,0,2020-02-16T14:00:00Z,1920,1.0
82806,Ravel_Morrison,26,3,0,Sheffield United,Bournemouth,0.293816,0.704973,True,0,...,1,2,0.0,-322,11,333,0,2020-02-09T14:00:00Z,1920,0.0


In [6]:
# find the latest gameweek
last_gw = df_train['gw'][df_train['season'] == '1920'].max()

if np.isnan(last_gw): 
    last_gw = 1 
else: 
    last_gw = last_gw + 1
    
last_gw

27

In [7]:
# build training set for latest gameweek
df_latest = build_season(season_paths[0], season_names[0], all_players, teams, teams_mv, [last_gw])

In [8]:
df_latest
#df_latest[df_latest['player'] == 'Kevin_De Bruyne']

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,selected,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season
0,Chris_Mepham,27,2,0,Bournemouth,Burnley,0.704973,0.402621,False,0,...,48161,0,3,0.0,-1023,10,1033,0,2020-02-22T15:00:00Z,1920
1,Paulo_Gazzaniga,27,1,0,Tottenham Hotspur,Chelsea,1.845990,1.744281,False,0,...,169986,1,2,0.0,-24361,1481,25842,0,2020-02-22T12:30:00Z,1920
2,John_Egan,27,2,90,Sheffield United,Brighton and Hove Albion,0.293816,0.467411,True,2,...,166709,1,1,25.0,29929,35357,5428,0,2020-02-22T15:00:00Z,1920
3,Max_Meyer,27,3,0,Crystal Palace,Newcastle United,0.473067,0.640348,True,0,...,5878,0,1,0.0,-7,38,45,0,2020-02-22T15:00:00Z,1920
4,Fabian_Delph,27,2,81,Everton,Arsenal,0.978426,1.437198,False,1,...,13428,2,3,8.0,-179,86,265,0,2020-02-23T16:30:00Z,1920
5,James_Justin,27,2,0,Leicester City,Manchester City,1.030978,2.653289,True,0,...,7425,1,0,0.0,-20,51,71,0,2020-02-22T17:30:00Z,1920
6,Che_Adams,27,4,17,Southampton,Aston Villa,0.452499,0.537651,True,4,...,23541,0,2,27.0,-44,346,390,0,2020-02-22T15:00:00Z,1920
7,Mahmoud Ahmed_Ibrahim Hassan,27,3,62,Aston Villa,Southampton,0.537651,0.452499,False,2,...,83123,0,2,21.0,-1883,1041,2924,0,2020-02-22T15:00:00Z,1920
8,Tyreece_John-Jules,27,4,0,Arsenal,Everton,1.437198,0.978426,True,0,...,356,2,3,0.0,-9,37,46,0,2020-02-23T16:30:00Z,1920
9,Adrian_Mariappa,27,2,0,Watford,Manchester United,0.476769,1.646993,False,0,...,26584,0,3,0.0,-1625,890,2515,0,2020-02-23T14:00:00Z,1920


In [9]:
df_latest.shape

(624, 34)

In [10]:
last_play_proba = remaining_season[remaining_season['gw'] == last_gw][['player', 'play_proba']]

In [11]:
last_play_proba.head()

,player,play_proba
0,Shkodran_Mustafi,1.00
1,Héctor_Bellerín,1.00
2,Sead_Kolasinac,1.00
3,Ainsley_Maitland-Niles,1.00
4,Sokratis_Papastathopoulos,0.75


In [12]:
# add latest week's play_proba for each player
df_latest = df_latest.merge(last_play_proba, on='player', how='left')

In [13]:
df_latest

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
0,Chris_Mepham,27,2,0,Bournemouth,Burnley,0.704973,0.402621,False,0,...,0,3,0.0,-1023,10,1033,0,2020-02-22T15:00:00Z,1920,0.0
1,Paulo_Gazzaniga,27,1,0,Tottenham Hotspur,Chelsea,1.845990,1.744281,False,0,...,1,2,0.0,-24361,1481,25842,0,2020-02-22T12:30:00Z,1920,1.0
2,John_Egan,27,2,90,Sheffield United,Brighton and Hove Albion,0.293816,0.467411,True,2,...,1,1,25.0,29929,35357,5428,0,2020-02-22T15:00:00Z,1920,1.0
3,Max_Meyer,27,3,0,Crystal Palace,Newcastle United,0.473067,0.640348,True,0,...,0,1,0.0,-7,38,45,0,2020-02-22T15:00:00Z,1920,1.0
4,Fabian_Delph,27,2,81,Everton,Arsenal,0.978426,1.437198,False,1,...,2,3,8.0,-179,86,265,0,2020-02-23T16:30:00Z,1920,1.0
5,James_Justin,27,2,0,Leicester City,Manchester City,1.030978,2.653289,True,0,...,1,0,0.0,-20,51,71,0,2020-02-22T17:30:00Z,1920,1.0
6,Che_Adams,27,4,17,Southampton,Aston Villa,0.452499,0.537651,True,4,...,0,2,27.0,-44,346,390,0,2020-02-22T15:00:00Z,1920,1.0
7,Mahmoud Ahmed_Ibrahim Hassan,27,3,62,Aston Villa,Southampton,0.537651,0.452499,False,2,...,0,2,21.0,-1883,1041,2924,0,2020-02-22T15:00:00Z,1920,1.0
8,Tyreece_John-Jules,27,4,0,Arsenal,Everton,1.437198,0.978426,True,0,...,2,3,0.0,-9,37,46,0,2020-02-23T16:30:00Z,1920,0.0
9,Adrian_Mariappa,27,2,0,Watford,Manchester United,0.476769,1.646993,False,0,...,0,3,0.0,-1625,890,2515,0,2020-02-23T14:00:00Z,1920,1.0


In [14]:
df_train_new = pd.concat([df_train, df_latest], ignore_index=True, axis=0)

In [15]:
df_train_new.tail()

,player,gw,position,minutes,team,opponent_team,relative_market_value_team,relative_market_value_opponent_team,was_home,total_points,...,team_a_score,team_h_score,threat,transfers_balance,transfers_in,transfers_out,yellow_cards,kickoff_time,season,play_proba
83426,Heurelho da Silva_Gomes,27,1,0,Watford,Manchester United,0.476769,1.646993,False,0,...,0,3,0.0,-60,96,156,0,2020-02-23T14:00:00Z,1920,1.0
83427,Joseph_Anang,27,1,0,West Ham United,Liverpool,0.758450,2.447608,False,0,...,2,3,0.0,28,382,354,0,2020-02-24T20:00:00Z,1920,1.0
83428,Erik_Pieters,27,2,0,Burnley,Bournemouth,0.402621,0.704973,True,0,...,0,3,0.0,-5232,2356,7588,0,2020-02-22T15:00:00Z,1920,1.0
83429,Japhet_Tanganga,27,2,90,Tottenham Hotspur,Chelsea,1.845990,1.744281,False,1,...,1,2,0.0,1959,18209,16250,0,2020-02-22T12:30:00Z,1920,1.0
83430,Ravel_Morrison,27,3,0,Sheffield United,Brighton and Hove Albion,0.293816,0.467411,True,0,...,1,1,0.0,-240,8,248,0,2020-02-22T15:00:00Z,1920,0.0


In [16]:
# save latest training set to csv
# overwrite existing one
df_train_new.to_csv(path/'train.csv')

In [17]:
## now need to create the prediction set
# start by reading fixtures.csv
# set starting gameweek (where are we right now in the season)
current_gw = last_gw + 1
fixtures = pd.read_csv(path/'fixtures.csv')
fixtures = fixtures[fixtures['gw'] >= current_gw]

In [18]:
fixtures.head(10)

,home_team,away_team,gw
270,Aston Villa,Sheffield United,34
271,Bournemouth,Chelsea,28
272,Brighton and Hove Albion,Crystal Palace,28
273,Everton,Manchester United,28
274,Manchester City,Arsenal,34
275,Newcastle United,Burnley,28
276,Norwich,Leicester City,28
277,Tottenham Hotspur,Wolverhampton Wanderers,28
278,Watford,Liverpool,28
279,West Ham United,Southampton,28


In [19]:
# add team codes for home and away teams
fixtures = fixtures.merge(teams, left_on='home_team', right_on='team', how='left')
fixtures = fixtures.merge(teams, left_on='away_team', right_on='team', how='left')
fixtures = fixtures[['gw', 'home_team', 'away_team', 'team_code_x', 'team_code_y']]
fixtures.rename(index=str,
                columns={'team_code_x':'home_team_code',
                         'team_code_y':'away_team_code'},
                inplace=True)

In [20]:
fixtures.head()

,gw,home_team,away_team,home_team_code,away_team_code
0,34,Aston Villa,Sheffield United,7,49
1,28,Bournemouth,Chelsea,91,8
2,28,Brighton and Hove Albion,Crystal Palace,36,31
3,28,Everton,Manchester United,11,1
4,34,Manchester City,Arsenal,43,3


In [21]:
# join home team to all players for current season
home_df = fixtures.merge(all_players, 
               left_on='home_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
home_df = home_df[['gw', 'home_team', 'away_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
home_df.rename(index=str, 
               columns={'home_team':'team',
                        'away_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
home_df['was_home'] = True

In [22]:
# join away team to all players for current season
away_df = fixtures.merge(all_players, 
               left_on='away_team_code', 
               right_on='team_1920', 
               how='left')

# pull out the required fields and rename columns
away_df = away_df[['gw',  'away_team', 'home_team', 'full_name', 'position_1920', 'cost_1920', 'play_proba_1920']]
away_df.rename(index=str, 
               columns={'away_team':'team',
                        'home_team':'opponent_team',
                        'full_name':'player',
                        'position_1920':'position',
                        'cost_1920':'price',
                        'play_proba_1920':'play_proba'},
              inplace=True)

# add home flag
away_df['was_home'] = False

In [23]:
# look at away players
away_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,34,Sheffield United,Aston Villa,Lys_Mousset,4,46,100,False
1,34,Sheffield United,Aston Villa,Enda_Stevens,2,51,75,False
2,34,Sheffield United,Aston Villa,Kieron_Freeman,2,42,None,False
3,34,Sheffield United,Aston Villa,Jack_O'Connell,2,47,None,False
4,34,Sheffield United,Aston Villa,George_Baldock,2,51,None,False


In [24]:
# look at home players
home_df.head()

,gw,team,opponent_team,player,position,price,play_proba,was_home
0,34,Aston Villa,Sheffield United,Ahmed_El Mohamady,2,42,None,True
1,34,Aston Villa,Sheffield United,James_Chester,2,43,0,True
2,34,Aston Villa,Sheffield United,Neil_Taylor,2,43,None,True
3,34,Aston Villa,Sheffield United,Kortney_Hause,2,44,100,True
4,34,Aston Villa,Sheffield United,Jonathan_Kodjia,4,54,0,True


In [25]:
# calculate average minutes over last 5 gameweeks
# first get last 5 gameweeks
recent_mins = df_train_new[(df_train_new['gw'] >= current_gw - 5) & 
                           (df_train_new['season'] == season_names[0])][['player', 'minutes', 'play_proba']]

# exclude weeks where the player had an injury designation (i.e. keep where play_proba is 1)
recent_mins = recent_mins[recent_mins['play_proba'] == 1]

len(recent_mins)

# average for each player
recent_mins = recent_mins.groupby(['player'])['minutes'].mean().to_dict()

recent_mins

{'Aaron_Connolly': 22.4,
 'Aaron_Cresswell': 90.0,
 'Aaron_Lennon': 1.8,
 'Aaron_Mooy': 85.25,
 'Aaron_Ramsdale': 90.0,
 'Aaron_Wan-Bissaka': 90.0,
 'Abd-Al-Ali Morakinyo Olaposi_Koiki': 0.0,
 'Abdoulaye_Doucouré': 87.8,
 'Adam_Idah': 0.0,
 'Adam_Lallana': 3.4285714285714284,
 'Adam_Masina': 90.0,
 'Adam_Smith': 90.0,
 'Adam_Webster': 72.0,
 'Adama_Traoré': 68.25,
 'Adrian_Mariappa': 72.0,
 'Adrián_San Miguel del Castillo': 0.0,
 'Ahmed_El Mohamady': 0.0,
 'Ainsley_Maitland-Niles': 18.0,
 'Albian_Ajeti': 0.75,
 'Alex_Iwobi': 41.0,
 'Alex_McCarthy': 90.0,
 'Alex_Oxlade-Chamberlain': 66.875,
 'Alexander_Tettey': 85.8,
 'Alexandre_Lacazette': 51.2,
 'Alexis_Mac Allister': 0.0,
 'Alfie_Whiteman': 0.0,
 'Alireza_Jahanbakhsh': 15.2,
 'Alisson_Ramses Becker': 90.0,
 'Allan_Saint-Maximin': 64.5,
 'Andre_Gray': 1.8,
 'Andreas_Christensen': 81.0,
 'Andreas_Pereira': 39.8,
 'Andrew_Robertson': 90.0,
 'Andrew_Surman': 35.4,
 'Andros_Townsend': 4.333333333333333,
 'André Filipe_Tavares Gomes': 31.0

In [26]:
# concatenate home and away players
remaining_season_df = home_df.append(away_df).reset_index(drop=True)

# add home and away team market values
remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df = remaining_season_df.merge(teams_mv[teams_mv['season'] == '1920'],
                                                left_on='opponent_team', 
                                                right_on='name', 
                                                how='left').drop(['name', 'season'], axis=1)

remaining_season_df.rename(index=str, 
                           columns={'relative_market_value_x':'relative_market_value_team',
                                    'relative_market_value_y':'relative_market_value_opponent_team'},
                           inplace=True)

# add season name
remaining_season_df['season'] = '1920'

# divide cost by 10 for actual cost
remaining_season_df['price'] = remaining_season_df['price']/10

# set availability probability
# 0 = 0% chance, 25 = 25% chance, etc
# 'None' or '100' = 100% chance
remaining_season_df.loc[remaining_season_df['play_proba'] == 'None', 'play_proba'] = 100
remaining_season_df['play_proba'] = remaining_season_df['play_proba'].astype('float') / 100

# cast position to integer
remaining_season_df['position'] = remaining_season_df['position'].astype(int)

# add minutes based on last three fixtures
remaining_season_df['minutes'] = remaining_season_df['player'].map(recent_mins)
remaining_season_df['minutes'].fillna(0, inplace=True)

# multiply minutes by play probability
remaining_season_df['minutes'] = remaining_season_df['minutes'] * remaining_season_df['play_proba']



In [27]:
remaining_season_df.head(50)

,gw,team,opponent_team,player,position,price,play_proba,was_home,relative_market_value_team,relative_market_value_opponent_team,season,minutes
0,34,Aston Villa,Sheffield United,Ahmed_El Mohamady,2,4.2,1.00,True,0.537651,0.293816,1920,0.000000
1,34,Aston Villa,Sheffield United,James_Chester,2,4.3,0.00,True,0.537651,0.293816,1920,0.000000
2,34,Aston Villa,Sheffield United,Neil_Taylor,2,4.3,1.00,True,0.537651,0.293816,1920,0.000000
3,34,Aston Villa,Sheffield United,Kortney_Hause,2,4.4,1.00,True,0.537651,0.293816,1920,90.000000
4,34,Aston Villa,Sheffield United,Jonathan_Kodjia,4,5.4,0.00,True,0.537651,0.293816,1920,0.000000
5,34,Aston Villa,Sheffield United,Ørjan_Nyland,1,4.3,1.00,True,0.537651,0.293816,1920,0.000000
6,34,Aston Villa,Sheffield United,Conor_Hourihane,3,5.7,1.00,True,0.537651,0.293816,1920,7.000000
7,34,Aston Villa,Sheffield United,Jack_Grealish,3,6.6,1.00,True,0.537651,0.293816,1920,90.000000
8,34,Aston Villa,Sheffield United,Anwar_El Ghazi,3,5.5,0.75,True,0.537651,0.293816,1920,49.950000
9,34,Aston Villa,Sheffield United,John_McGinn,3,5.4,0.00,True,0.537651,0.293816,1920,0.000000


In [28]:
# save latest prediction set to csv
remaining_season_df.to_csv(path/'remaining_season.csv')